# Stacked Generalizations

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.svm as skl_svm
import sklearn.cross_validation as skl_cv
import seaborn as sns
import os
import sys

base_path = '/home/lundi/Python/MNIST/'
sys.path.append(base_path + '/libraries/')

import time
import glob

import MNIST_model_functions as mmf
MNIST_model_functions = mmf.MNIST_model_functions()

import sklearn.linear_model as skl_lm
import sklearn.grid_search as skl_gs

In [2]:
prediction_data_v1 = pd.DataFrame()

for filename in glob.glob(base_path + '/data/prediction_results/2016.11.7-*_results.csv'):
    prediction_data_v1 = pd.concat([prediction_data_v1, pd.read_csv(filename)])
prediction_data_v1 = prediction_data_v1.rename(columns = {'Unnamed: 0': 'datum_index'})
#prediction_data_v1 = prediction_data_v1.drop(['Unnamed: 0'], axis = 1)

In [3]:
X = pd.pivot_table(prediction_data_v1, index = 'datum_index', values=['0','1','2','3','4','5','6','7','8','9'], columns=['Model'])
X.columns = ['%s%s' % (a, '|%s' % b if b else '') for a, b in X.columns]

In [8]:
y = prediction_data_v1[['datum_index','Actual']]
y = y.drop_duplicates().set_index('datum_index')
y['Actual'] = y['Actual'].astype(int)
y = y['Actual']

Creating a linear model

In [9]:
lr_gs_clf = skl_gs.GridSearchCV(skl_lm.LogisticRegression(), param_grid=[
        {'C': np.logspace(-10,10,num=30)}
        ], cv = 5)

lr_gs_clf.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': array([  1.00000e-10,   4.89390e-10,   2.39503e-09,   1.17210e-08,
         5.73615e-08,   2.80722e-07,   1.37382e-06,   6.72336e-06,
         3.29034e-05,   1.61026e-04,   7.88046e-04,   3.85662e-03,
         1.88739e-02,   9.23671e-02,   4.52035e-01,   2.21222e+00,
         1.082...   3.56225e+06,   1.74333e+07,   8.53168e+07,   4.17532e+08,
         2.04336e+09,   1.00000e+10])}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [10]:
for params, mean_score, scores in lr_gs_clf.grid_scores_:
    print mean_score, params

0.941811637672 {'C': 1e-10}
0.941811637672 {'C': 4.8939009184774994e-10}
0.941811637672 {'C': 2.3950266199874909e-09}
0.941811637672 {'C': 1.1721022975334793e-08}
0.941811637672 {'C': 5.7361525104486813e-08}
0.941611677664 {'C': 2.8072162039411812e-07}
0.941611677664 {'C': 1.3738237958832638e-06}
0.941611677664 {'C': 6.7233575364993352e-06}
0.941811637672 {'C': 3.290344562312671e-05}
0.94201159768 {'C': 0.00016102620275609426}
0.94301139772 {'C': 0.00078804628156699209}
0.943411317736 {'C': 0.0038566204211634724}
0.94401119776 {'C': 0.018873918221350997}
0.945610877824 {'C': 0.092367085718738848}
0.948210357928 {'C': 0.45203536563602403}
0.952209558088 {'C': 2.2122162910704501}
0.95400919816 {'C': 10.826367338740562}
0.955208958208 {'C': 52.983169062837234}
0.952809438112 {'C': 259.29437974046778}
0.952409518096 {'C': 1268.9610031679233}
0.951809638072 {'C': 6210.1694189156287}
0.951409718056 {'C': 30391.953823132073}
0.951409718056 {'C': 148735.21072935118}
0.951209758048 {'C': 727895

In [12]:
lr_clf = skl_lm.LogisticRegression(C = 50)
skl_cv.cross_val_score(lr_clf, X = X, y = y, cv = 10).mean()

0.95601063561760447

In [13]:
y_stacked_pred_cv = skl_cv.cross_val_predict(lr_clf, X = X, y = y, cv = 10)

In [15]:
data_to_output = X.copy()
data_to_output['Prediction_Stacked'] = y_stacked_pred_cv
data_to_output['Actual'] = y

In [25]:
data_to_output.to_csv('./../../data/prediction_results/2016.11.9-Stacked_Results.csv')

In [18]:
lr_clf.fit(X, y)

LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)